In [1]:
import numpy as np
import cv2
import scipy.io as sio
# from pathlib import Path
import pandas as pd
import mediapipe
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
import os

# get list of images in the directory
images = [ image[:-4] for image in os.listdir('./AFLW2000') if image.endswith('.jpg')]


# Preparing Data

In [3]:
faces_landmarks = {}
faceModule = mediapipe.solutions.face_mesh
with faceModule.FaceMesh(static_image_mode=True) as faces:
    for image_name in images:
        try:
            image = cv2.imread('./AFLW2000/'+image_name+'.jpg')

            results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results.multi_face_landmarks != None: 
                
                mat_file = sio.loadmat('./AFLW2000/'+image_name+'.mat')
                pose_para = mat_file["Pose_Para"][0][:3]
                pitch = pose_para[0]
                yaw = pose_para[1]
                roll = pose_para[2]

                faces_landmarks[image_name] = {'pitch':pitch,'yaw':yaw,'roll':roll}
            # looping over the faces in the image
                for face in results.multi_face_landmarks:
                    # for landmark in face.landmark:
                    for index, landmark in enumerate(face.landmark):
                        # if index in selected_indices:
                        faces_landmarks[image_name]["x_"+str(index)] = landmark.x * image.shape[1]
                        faces_landmarks[image_name]["y_"+str(index)] = landmark.y * image.shape[0]
            
        except:
            pass

df = pd.DataFrame.from_dict(faces_landmarks,orient='index')
df = df.reset_index()
df = df.rename(columns={'index':'image_name'})

In [4]:
df

,image_name,pitch,yaw,roll,x_0,y_0,x_1,y_1,x_2,y_2,...,x_463,y_463,x_464,y_464,x_465,y_465,x_466,y_466,x_467,y_467
0,image00002,-0.399231,0.018227,0.085676,218.542837,309.100825,220.137329,287.874761,220.411143,291.265529,...,251.870477,218.311456,246.796650,221.765859,243.642935,225.167128,299.550927,212.101270,304.240984,208.802274
1,image00004,0.470065,1.189533,0.300959,198.675755,288.941020,187.772736,266.513032,201.977581,274.745643,...,235.110319,228.130272,230.126774,230.275959,225.295928,231.711021,263.002905,225.110024,267.327082,222.924252
2,image00006,-0.184650,0.881137,-0.236852,143.008733,359.635493,144.597419,349.424055,143.242085,351.929694,...,150.485645,323.007488,148.650210,324.390677,147.887711,325.504062,168.031865,320.881972,169.939251,319.409144
3,image00008,-0.175379,0.299208,-0.373374,226.764786,312.438914,211.284471,294.260994,220.276603,296.437794,...,228.405172,225.431138,223.978803,229.772723,220.763491,233.393463,268.026790,205.024758,271.201319,200.080495
4,image00013,-0.026812,0.011965,-0.220662,229.607284,301.067019,221.727973,272.158626,224.890137,281.010506,...,234.225270,216.860247,229.834494,219.893289,227.550003,221.706301,271.117607,203.714633,275.089368,201.859349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,image04345,-0.306358,-0.283822,0.038554,231.949255,307.777798,239.776450,281.603032,233.466661,287.948093,...,247.130638,223.704842,243.843618,226.453462,243.032920,228.780654,283.077067,215.875420,286.094338,213.582815
1849,image04348,-0.367547,-0.429723,0.122791,244.757983,310.430047,254.378557,285.878098,247.179750,291.075361,...,261.945230,221.897368,259.196475,225.114530,258.738139,227.907327,296.910077,215.917705,300.489405,211.899367
1850,image04356,-0.156035,0.567114,-0.108536,263.868052,306.046137,271.983236,282.465202,267.206919,290.631670,...,298.735777,241.784969,294.392154,242.765021,292.006141,243.641755,330.307844,246.450752,334.477633,243.062451
1851,image04358,-0.197102,-0.070430,0.105118,221.227473,311.809668,225.281632,286.933842,223.777047,292.424726,...,256.836560,220.644616,250.963730,223.467118,247.625345,226.170892,303.283355,216.844381,308.066538,213.429111


# Select specific landmarks

In [5]:
selected_indices = [0, 1, 4, 5, 6, 8, 9, 10, 280, 50, 123, 352, 164, 225, 359, 173, 398, 152, 185, 270, 434, 214, 172, 364]

def get_selected_features(df, selected_indices, default_features = []):
    list = default_features
    for i in sorted(selected_indices):
        list.append('x_'+str(i))
        list.append('y_'+str(i))
    return df.loc[:,list]


selected_df = get_selected_features(df, selected_indices, default_features = ['image_name','pitch','yaw','roll'])

# Normalize the landmarks

In [6]:
# center all points around the nose
def normalize_landmarks(df, selected_indices=np.arange(0,468,1),nose_index=5 , chain_index=152):
    nose_x = df['x_'+str(nose_index)]
    nose_y = df['y_'+str(nose_index)]

    for i in selected_indices:
        df['x_'+str(i)] = df['x_'+str(i)] - nose_x
        df['y_'+str(i)] = df['y_'+str(i)] - nose_y


    # scale all the points by the distance from the nose to the chin
    chin_x = df['x_'+str(chain_index)]
    chin_y = df['y_'+str(chain_index)]
    distance = np.sqrt(chin_x**2 + chin_y**2)

    for i in selected_indices:
        df['x_'+str(i)] = df['x_'+str(i)] / distance
        df['y_'+str(i)] = df['y_'+str(i)] / distance
    
    return df

df = normalize_landmarks(selected_df, selected_indices)
df


,image_name,pitch,yaw,roll,x_0,y_0,x_1,y_1,x_4,y_4,...,x_352,y_352,x_359,y_359,x_364,y_364,x_398,y_398,x_434,y_434
0,image00002,-0.399231,0.018227,0.085676,-0.033682,0.405341,-0.018732,0.206318,-0.012975,0.128226,...,0.938155,-0.074850,0.767072,-0.492689,0.657925,0.562671,0.341639,-0.483363,0.628707,0.465324
1,image00004,0.470065,1.189533,0.300959,0.072360,0.409648,-0.046729,0.164675,-0.054181,0.089544,...,1.093607,0.156688,0.845935,-0.264811,0.960750,0.753903,0.514256,-0.279008,0.799796,0.613322
2,image00006,-0.184650,0.881137,-0.236852,-0.021531,0.369416,0.008212,0.178240,0.009319,0.105100,...,0.651442,-0.027367,0.483795,-0.348917,0.487496,0.548927,0.161886,-0.339782,0.471607,0.428797
3,image00008,-0.175379,0.299208,-0.373374,0.204306,0.366771,0.054153,0.190453,0.020258,0.119304,...,1.004375,-0.399094,0.660144,-0.680999,1.074779,0.222056,0.266629,-0.529130,0.945926,0.153862
4,image00013,-0.026812,0.011965,-0.220662,0.096428,0.401257,0.033057,0.168756,0.016667,0.096024,...,0.721539,-0.058329,0.474700,-0.364687,0.712596,0.524699,0.168019,-0.315833,0.632333,0.408939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,image04345,-0.306358,-0.283822,0.038554,-0.077121,0.458112,-0.000002,0.200220,0.011110,0.119140,...,0.558285,-0.053425,0.446727,-0.429504,0.318976,0.544572,0.115145,-0.410396,0.345390,0.450901
1849,image04348,-0.367547,-0.429723,0.122791,-0.094065,0.424305,-0.005930,0.199382,0.007292,0.120181,...,0.483437,-0.064868,0.403035,-0.435805,0.242383,0.523317,0.101957,-0.417651,0.274876,0.431476
1850,image04356,-0.156035,0.567114,-0.108536,-0.109277,0.362854,-0.037823,0.155223,-0.017276,0.086784,...,0.530578,0.206899,0.498912,-0.145636,0.270893,0.727530,0.242264,-0.209157,0.288310,0.598761
1851,image04358,-0.197102,-0.070430,0.105118,-0.062085,0.446608,-0.024317,0.214868,-0.012612,0.129998,...,0.871431,0.000920,0.744135,-0.422188,0.559635,0.625977,0.329120,-0.436322,0.545910,0.527042


# Save the dataset in csv file

In [7]:
df.to_csv('AFLW2000.csv')